In [2]:
import os
import re
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

In [3]:
#user setup area
DWT_limit = 5000 #change this for the cap (multiplier of 1000)
vessel_type = 402 #401 bulk carrier , 402 general cargo , 403 container , 6 all tanker 
minYear = '1985'
maxYear = '2022'

In [4]:
filename = 'generalcargo.csv'
directory ='C:/Users/abinp/Documents/ITS/side projects/new/imo scrapping/output/'
csv_write = '{}{}'.format(directory,filename)

In [5]:
%ls
link =''
vessel_names = []
minDW = -1000
maxDW = 0

 Volume in drive C has no label.
 Volume Serial Number is A071-85E6

 Directory of C:\Users\abinp\Documents\ITS\BANGKIT\capstone\ML-Bangkit-Capstone-Project\GetData\scrapping review

23/05/2023  06:29    <DIR>          .
21/05/2023  21:48    <DIR>          ..
21/05/2023  21:59    <DIR>          .ipynb_checkpoints
25/04/2023  08:54            10.526 contekan.ipynb
22/05/2023  15:40           189.122 Imo kapal.ipynb
21/05/2023  22:03    <DIR>          nama
21/05/2023  22:02    <DIR>          response
21/05/2023  22:05             3.123 scrapping nama.ipynb
23/05/2023  06:29             6.990 scrapping review.ipynb
               4 File(s)        209.761 bytes
               5 Dir(s)  86.431.379.456 bytes free


In [7]:
def changelink(min,max):
    global minDW , maxDW , link
    minDW = min + 1000
    maxDW = max + 1000
    link = "https://www.vesselfinder.com/vessels?type={}&minDW={}&maxDW={}&minYear={}&maxYear={}&dir=1&sort=4".format(vessel_type,minDW,maxDW,minYear,maxYear)   
changelink(minDW,maxDW)
print(link)

https://www.vesselfinder.com/vessels?type=402&minDW=1000&maxDW=2000&minYear=1985&maxYear=2022&dir=1&sort=4


In [17]:
# create webdriver instance using options
driver = webdriver.Chrome(executable_path=r"chromedriver")

# navigate to the link
driver.get(link)

In [18]:
while True:
    kelas = driver.find_elements(By.CLASS_NAME, "ship-link")
    for element in kelas:
        link = element.get_attribute("href")
        number = re.findall(r'\d+', link)[0]
        vessel_names.append(number)
    try:
        next_page = driver.find_element(By.CLASS_NAME, "pagination-next")
        if not next_page.is_enabled():
            print("Out of pages")
            break
        next_page.click()
        
    except NoSuchElementException:
        if maxDW == DWT_limit:
            print("reaching end of range")
            break
        else:
            changelink(minDW,maxDW)
            print("changing range |  min dw :{}   max dw: {}".format(minDW,maxDW))
            driver = webdriver.Chrome(executable_path=r"chromedriver")
            driver.get(link)
            continue

KeyboardInterrupt: 

In [70]:
duplicates = []
for name in vessel_names:
    if vessel_names.count(name) > 1:
        duplicates.append(name)

duplicates = list(set(duplicates))

for name in duplicates:
    print("Removing duplicates:", name)
    
#clear duplicate
clear_list = list(set(vessel_names))
vessel_names = clear_list

Removing duplicates: 9646845
Removing duplicates: 9008677
Removing duplicates: 8672627
Removing duplicates: 9932995
Removing duplicates: 9842982
Removing duplicates: 9957309
Removing duplicates: 9738181
Removing duplicates: 8535776
Removing duplicates: 9990923
Removing duplicates: 9540039
Removing duplicates: 8632433
Removing duplicates: 9698850
Removing duplicates: 9281700
Removing duplicates: 9934187
Removing duplicates: 9515230
Removing duplicates: 9606118
Removing duplicates: 9623532
Removing duplicates: 9603972
Removing duplicates: 9824758
Removing duplicates: 9433107
Removing duplicates: 9919917
Removing duplicates: 9952414
Removing duplicates: 9632935
Removing duplicates: 9958212
Removing duplicates: 9506320
Removing duplicates: 8357887
Removing duplicates: 9540041
Removing duplicates: 9925746
Removing duplicates: 9948308
Removing duplicates: 9637595
Removing duplicates: 9634816
Removing duplicates: 9824734
Removing duplicates: 9798923
Removing duplicates: 9746970
Removing dupli

In [71]:
print(vessel_names)

['9474230', '9493420', '9633288', '9343845', '9336945', '9479230', '9263253', '9587984', '9368869', '9581136', '9546095', '9849318', '9732943', '9550266', '9675949', '9403205', '9837298', '9919644', '9220990', '9424132', '8671477', '9157349', '9909089', '9558086', '9620530', '9425461', '9418731', '9633006', '9473846', '9840506', '8804098', '9077367', '9415155', '9528861', '9267211', '8710326', '9815305', '9888015', '9614385', '9628128', '9855599', '9443774', '9646613', '9483499', '8806486', '9754161', '9835941', '9603518', '9701803', '9321926', '9594456', '9346847', '9548653', '9595175', '9329825', '9298533', '9200653', '9132686', '9624469', '9552317', '9797682', '9118329', '8407682', '9558907', '9707675', '9838450', '9922005', '9941544', '9109902', '9492098', '9143087', '9762601', '9300843', '9136539', '9630743', '9782182', '8517580', '9115092', '9459981', '9659737', '9438054', '9475545', '9591301', '9605009', '9450703', '9694995', '9510539', '9790842', '8913980', '9932127', '9243564'

In [72]:
len(vessel_names)
#target : 13587 ships

13587

In [73]:
# assume the list of names is stored in a variable called "vessel_names"

# open a new CSV file for writing
with open(csv_write, 'w', newline='') as csvfile:
    # create a CSV writer object
    writer = csv.writer(csvfile)
    
    # write the header row (optional)
    writer.writerow(['IMO'])

    # write each name and its corresponding minDW and maxDW values to a new row in the CSV file
    for i in range(0, len(vessel_names)):
        name = vessel_names[i]
        writer.writerow([name])
